In [6]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
import detectron2

from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.structures import BoxMode

# データセットがCOCOフォーマットなら以下のコードは次の3行で置き換えることができる。
from detectron2.data.datasets import register_coco_instances

# register_coco_instances("fox_falco_train", {}, "/home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_train.json", "/home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_train")
# register_coco_instances("fox_falco_val", {}, "/home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_val.json", "/home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_val")


1.6.0 True
gcc (Ubuntu 5.5.0-12ubuntu1~16.04) 5.5.0 20171010
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



AssertionError: Dataset 'fox_falco_train' is already registered!

In [3]:
from detectron2.modeling import build_model
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
import detectron2

from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.structures import BoxMode

from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import ColorMode
from detectron2.structures import Boxes

import tqdm
import time

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # falco, fox, smallBF
cfg.MODEL.WEIGHTS = "/home/tappun/wip/output/model_final.pth"
cfg.MODEL.DEVICE='cuda:1'

cfg.MODEL.ROI_HEADS.SCORE_THRESHOLD_TEST = 0.65
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.65
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.65
cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = 0.65

model = build_model(cfg)

predictor = DefaultPredictor(cfg)
# dataset_dicts = DatasetCatalog.get("fox_falco_val_2")

#ワンチャンGPUで早くなるかも


# video = cv2.VideoCapture('/home/tappun/short_fox_falco.mp4')
video = cv2.VideoCapture('/home/tappun/ssbu_dataset/movie/fox_falco_smallBF1.mp4')

print(video)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = video.get(cv2.CAP_PROP_FPS)
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize video writer
video_writer = cv2.VideoWriter('totemonagai2.mp4', fourcc=cv2.VideoWriter_fourcc(*"mp4v"), fps=float(frames_per_second), frameSize=(width, height), isColor=True)

v = VideoVisualizer(MetadataCatalog.get("fox_falco_val_2").set(thing_classes=["Falco", "Fox", "SmallBattleField"]),ColorMode.IMAGE)


def runOnVideo(video, maxFrames):
    """ Runs the predictor on every frame in the video (unless maxFrames is given),
    and returns the frame with the predictions drawn.
    """

    readFrames = 0
    while True:
        
        hasFrame, frame = video.read()
        
        if not hasFrame:
            print("動画の読み込みに失敗")
            break

        # Get prediction results for this frame
        outputs = predictor(frame)

        # Make sure the frame is colored
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        # Draw a visualization of the predictions using the video visualizer
        visualization = v.draw_instance_predictions(frame, outputs["instances"].to("cpu"))

        # Convert Matplotlib RGB format to OpenCV BGR format
        visualization = cv2.cvtColor(visualization.get_image(), cv2.COLOR_RGB2BGR)

        yield visualization

        readFrames += 1
        if readFrames > maxFrames:
            print("obobo")
            break
            
# metadata=MetadataCatalog.get("fox_falco_val_2").set(thing_classes=["Falco", "Fox", "PokemonStadium2"]),

# Create a cut-off for debugging
num_frames = 3600

# Enumerate the frames of the video
for visualization in tqdm.tqdm(runOnVideo(video, num_frames), total=num_frames):
    # Write test image
    cv2.imwrite('POSE detectron2.png', visualization)
    # Write to video file
    video_writer.write(visualization)

# Release resources
video.release()
video_writer.release()
# cv2.destroyAllWindows()

  0%|          | 0/3600 [00:00<?, ?it/s]

<VideoCapture 0x7f391c435c10>


3601it [13:54,  4.31it/s]                          

obobo


In [10]:

def train_dataset(dataset_name, dataset_name_val):
    cfg = get_cfg()
    cfg.MODEL.DEVICE='cuda:1'
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.DATASETS.TRAIN = (dataset_name,)
    cfg.DATASETS.TEST = (dataset_name_val,)
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
    cfg.SOLVER.MAX_ITER = 1000   # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256  # faster, and good enough for this toy dataset (default: 512)
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # only has one class (ballon)
    cfg.MODEL.ROI_HEADS.SCORE_THRESHOLD_TEST = 0.7

    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    trainer = DefaultTrainer(cfg) 
    trainer.resume_or_load(resume=False)
    
    print(dataset_name)
    
    
    
#     trainer.train()

    
if __name__ == '__main__':
    # Prepare the data set
    dataset_name = "fox_falco_train"
    dataset_name_val = "fox_falco_val"
#     dicts_name   = "balloon/train"
#     for d in ["train", "val"]:
#         DatasetCatalog.register("balloon_" + d, lambda d=d: get_balloon_dicts("balloon/" + d))
#         MetadataCatalog.get("balloon_" + d).set(thing_classes=["balloon"])
#     balloon_metadata = MetadataCatalog.get(dataset_name)
#     # Verify the data set
#     verify_dataset(dicts_name)

    # train
    train_dataset(dataset_name, dataset_name_val)

[12/03 21:22:37 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

WARNING [12/03 21:22:37 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/03 21:22:37 d2.data.datasets.coco]: Loaded 448 images in COCO format from /home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_train.json
[12/03 21:22:37 d2.data.build]: Removed 0 images with no usable annotations. 448 images left.
[12/03 21:22:37 d2.data.common]: Serializing 448 elements to byte tensors and concatenating them all ...
[12/03 21:22:37 d2.data.common]: Serialized dataset takes 0.24 MiB
[12/03 21:22:37 d2.data.dataset_mapper]: Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[12/03 21:22:37 d2.data.build]: Using training sampler TrainingSampler


Unable to load 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model!
Unable to load 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model!
Unable to load 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model!
Unable to load 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model!
Unable to load 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) in the model!
Unable to load 'roi_heads.mask_head.predictor.bias' to the model due to incompatible shapes: (80,) in the checkpoint but (3,) in the model!


fox_falco_train


In [9]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

dataset_name = "fox_falco_train"
dataset_name_val = "fox_falco_val"

cfg = get_cfg()
cfg.MODEL.DEVICE='cuda:1'
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (dataset_name,)
cfg.DATASETS.TEST = (dataset_name_val,)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 1000   # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256  # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # only has one class (ballon)
cfg.MODEL.ROI_HEADS.SCORE_THRESHOLD_TEST = 0.7

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)

evaluator = COCOEvaluator(dataset_name, cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, dataset_name)
inference_on_dataset(trainer.model, val_loader, evaluator)

[12/03 21:13:02 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

WARNING [12/03 21:13:02 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/03 21:13:02 d2.data.datasets.coco]: Loaded 448 images in COCO format from /home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_train.json
[12/03 21:13:02 d2.data.build]: Removed 0 images with no usable annotations. 448 images left.
[12/03 21:13:02 d2.data.common]: Serializing 448 elements to byte tensors and concatenating them all ...
[12/03 21:13:02 d2.data.common]: Serialized dataset takes 0.24 MiB
[12/03 21:13:02 d2.data.dataset_mapper]: Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[12/03 21:13:02 d2.data.build]: Using training sampler TrainingSampler


Unable to load 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model!
Unable to load 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model!
Unable to load 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model!
Unable to load 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model!
Unable to load 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) in the model!
Unable to load 'roi_heads.mask_head.predictor.bias' to the model due to incompatible shapes: (80,) in the checkpoint but (3,) in the model!


WARNING [12/03 21:13:02 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/03 21:13:02 d2.data.datasets.coco]: Loaded 448 images in COCO format from /home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_train.json
[12/03 21:13:02 d2.data.common]: Serializing 448 elements to byte tensors and concatenating them all ...
[12/03 21:13:02 d2.data.common]: Serialized dataset takes 0.24 MiB
[12/03 21:13:02 d2.data.dataset_mapper]: Augmentations used in training: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/03 21:13:02 d2.evaluation.evaluator]: Start inference on 448 images


/home/tappun/.pyenv/versions/3.6.8/lib/python3.6/site-packages/detectron2/layers/wrappers.py:226: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  return x.nonzero().unbind(1)


[12/03 21:13:07 d2.evaluation.evaluator]: Inference done 11/448. 0.1083 s / img. ETA=0:02:11
[12/03 21:13:12 d2.evaluation.evaluator]: Inference done 28/448. 0.1060 s / img. ETA=0:02:05
[12/03 21:13:17 d2.evaluation.evaluator]: Inference done 45/448. 0.1057 s / img. ETA=0:01:59
[12/03 21:13:22 d2.evaluation.evaluator]: Inference done 62/448. 0.1057 s / img. ETA=0:01:54
[12/03 21:13:27 d2.evaluation.evaluator]: Inference done 79/448. 0.1056 s / img. ETA=0:01:49
[12/03 21:13:32 d2.evaluation.evaluator]: Inference done 97/448. 0.1052 s / img. ETA=0:01:43
[12/03 21:13:37 d2.evaluation.evaluator]: Inference done 114/448. 0.1052 s / img. ETA=0:01:38
[12/03 21:13:42 d2.evaluation.evaluator]: Inference done 131/448. 0.1053 s / img. ETA=0:01:33
[12/03 21:13:47 d2.evaluation.evaluator]: Inference done 148/448. 0.1054 s / img. ETA=0:01:28
[12/03 21:13:53 d2.evaluation.evaluator]: Inference done 166/448. 0.1055 s / img. ETA=0:01:23
[12/03 21:13:58 d2.evaluation.evaluator]: Inference done 184/448. 

OrderedDict([('bbox',
              {'AP': 0.008812550315104828,
               'AP50': 0.045465680600233246,
               'AP75': 0.0017669795612986852,
               'APs': nan,
               'APm': 0.007881242049787475,
               'APl': 0.027570920116389674,
               'AP-Falco': 0.01588134386513738,
               'AP-Fox': 0.010556307080177102,
               'AP-SmallBattleField': 0.0}),
             ('segm',
              {'AP': 9.579502011503834e-07,
               'AP50': 9.579502011503831e-06,
               'AP75': 0.0,
               'APs': nan,
               'APm': 0.0,
               'APl': 0.00015641374563996687,
               'AP-Falco': 0.0,
               'AP-Fox': 2.87385060345115e-06,
               'AP-SmallBattleField': 0.0})])